# Imports

In [1]:
from sklearn.decomposition import PCA
from pennylane import numpy as np
import pennylane as qml
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import combinations
from scipy import stats as st
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

# Function Definitions and Quantum Device Setup

In [2]:
def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])

In [3]:
def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))


adjoint_ansatz = qml.adjoint(ansatz)

In [4]:
def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    np.random.seed(5687)
    return np.random.uniform(0, 2 * np.pi, (num_layers, 2, num_wires), requires_grad=True)

In [5]:
dev = qml.device("default.qubit", wires=3, shots=None)
wires = dev.wires.tolist()

In [6]:
@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

In [7]:
def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

In [8]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [9]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [10]:
def train(params, X_train, y_train, steps, subset_size, lr,alignment_target=None, idx=None):
    opt = qml.GradientDescentOptimizer(lr)

    for i in range(steps):
        # Choose subset of datapoints to compute the KTA on.
        if idx:
            subset = idx
        else:
            subset = np.random.choice(list(range(len(X_train))), subset_size)
        # Define the cost function for optimization
        cost = lambda _params: -target_alignment(
          X_train[subset],
          y_train[subset],
          lambda x1, x2: kernel(x1, x2, _params),
          assume_normalized_kernel=True,
        )
        # Optimization step
        params = opt.step(cost, params)

        # Report the alignment on the full dataset every 50 steps.
        if (i + 1) % 50 == 0:
            current_alignment = target_alignment(
              X_train,
              y_train,
              lambda x1, x2: kernel(x1, x2, params),
              assume_normalized_kernel=True,
            )
            print(f"Step {i+1} - Alignment = {current_alignment:.3f}")
            if alignment_target and (current_alignment >= alignment_target):
                break
    return params

# IRIS L=3

In [13]:
from sklearn.datasets import load_iris
data = load_iris()

In [14]:
Y = np.copy(data['target'])
X = data['data']
pca = PCA(n_components=2)
X = pca.fit_transform(X)
scaler = MinMaxScaler()
X_minmax = scaler.fit_transform(X)*np.pi

In [15]:
quantum_accuracies_trained = []
quantum_accuracies_untrained = []
for rdm in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
    pred_all_trained = []
    pred_all_untrained = []
    all_params = {}
    init_params = random_params(num_wires=3, num_layers=3)
    init_kernel = lambda x1, x2: kernel(x1, x2, init_params)
    for y1, y2 in combinations(np.unique(Y).numpy(), r=2):
        mask = (y_train==y1) | (y_train==y2)
        y_train_pair = y_train[mask]
        X_train_pair = X_train[mask]
        y_train_pair [y_train_pair == y1] = -1
        y_train_pair [y_train_pair == y2] = 1
        svm_untrained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, init_kernel)).fit(X_train_pair, y_train_pair)
        test_mask = (y_test==y1) | (y_test==y2)
        y_test_pair = y_test[test_mask]
        X_test_pair = X_test[test_mask]
        y_test_pair [y_test_pair == y1] = -1
        y_test_pair [y_test_pair == y2] = 1
        y_pred_pair = svm_untrained.predict(X_test_pair)
        test_accuracy = 1 - np.count_nonzero(y_pred_pair - y_test_pair) / len(y_test_pair)
        print(f"Test Accuracy:{test_accuracy*100:.2f}")
        lr = 2
        new_test_accuracy = test_accuracy
        while (new_test_accuracy != 1) and (new_test_accuracy <= test_accuracy) and (lr >= 0.5):
            if lr >=1 :
                iterations= 500
            else:
                iterations= 1000
            params = all_params.get((y1, y2), init_params)
            params = train(params, X_train_pair, y_train_pair, iterations, 4, lr)
            all_params[(y1, y2)] = params
            trained_kernel = lambda x1, x2: kernel(x1, x2, all_params[(y1, y2)])
            svm_trained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)).fit(X_train_pair, y_train_pair)
            y_pred_pair_new = svm_trained.predict(X_test_pair)
            new_test_accuracy = 1 - np.count_nonzero(y_pred_pair_new - y_test_pair) / len(y_test_pair)
            print(f"Test Accuracy:{new_test_accuracy*100:.2f}")
            lr /= 2
        untrained_predictions = svm_untrained.predict(X_test)
        pred_all_untrained.append((untrained_predictions, (y1,y2)))
        if new_test_accuracy <= test_accuracy:
            pred_all_trained.append((untrained_predictions, (y1,y2)))
        else:
            pred_all_trained.append((svm_trained.predict(X_test), (y1,y2)))
    y_pred_all = []
    for y_pred_pair, labels in pred_all_untrained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_untrained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_untrained.append(final_accuracy_untrained*100)
    print(f"Final Accuracy Untrained {final_accuracy_untrained*100:.2f}")
    y_pred_all = []
    for y_pred_pair, labels in pred_all_trained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_trained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_trained.append(final_accuracy_trained*100)
    print(f"Final Accuracy Trained {final_accuracy_trained*100:.2f}")


Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Final Accuracy Untrained 100.00
Final Accuracy Trained 100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:93.33
Step 50 - Alignment = 0.303
Step 100 - Alignment = 0.311
Step 150 - Alignment = 0.319
Step 200 - Alignment = 0.300
Step 250 - Alignment = 0.302
Step 300 - Alignment = 0.302
Step 350 - Alignment = 0.305
Step 400 - Alignment = 0.309
Step 450 - Alignment = 0.301
Step 500 - Alignment = 0.309
Test Accuracy:93.33
Step 50 - Alignment = 0.309
Step 100 - Alignment = 0.318
Step 150 - Alignment = 0.322
Step 200 - Alignment = 0.313
Step 250 - Alignment = 0.313
Step 300 - Alignment = 0.324
Step 350 - Alignment = 0.316
Step 400 - Alignment = 0.323
Step 450 - Alignment = 0.312
Step 500 - Alignment = 0.335
Test Accuracy:93.33
Step 50 - Alignment = 0.327
Step 100 - Alignment = 0.330
Step 150 - Alignment = 0.328
Step 200 - Alignment = 0.332
Step 250 - Alignment = 0.331
Step 300 - Alignment = 0.334
Step 350 - Alignment =

In [16]:
np.array(quantum_accuracies_trained) >= np.array(quantum_accuracies_untrained)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True], requires_grad=True)

In [17]:
np.mean(quantum_accuracies_untrained)

94.44444444444444

In [18]:
np.std(quantum_accuracies_untrained)

3.0225490019412096

In [19]:
np.mean(quantum_accuracies_trained)

94.88888888888889

In [20]:
np.std(quantum_accuracies_trained)

2.4444444444444446

In [21]:
for ker in ['rbf', 'linear', 'poly']:
  acc = []
  for rdm in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
    clf = SVC(kernel=ker)
    clf.fit(X_train, y_train)
    acc.append(accuracy(clf, X_test, y_test)*100)
  print(f"Kernel: {ker}\t Mean: {np.mean(acc):.2f} \t Std: {np.std(acc):.2f}")

Kernel: rbf	 Mean: 94.67 	 Std: 2.67
Kernel: linear	 Mean: 94.67 	 Std: 2.47
Kernel: poly	 Mean: 95.11 	 Std: 2.59


# MNIST L=3

In [11]:
from sklearn.datasets import load_digits
digits = load_digits()

In [12]:
X = digits['data']
Y = digits['target']

idx = np.hstack([np.where(Y==i)[0][:20] for i in np.unique(Y)])
X = X[idx]
Y = Y[idx]

pca = PCA(n_components=2)
X = pca.fit_transform(X)
scaler = MinMaxScaler()
X_minmax = scaler.fit_transform(X)*np.pi

In [13]:
classical_accuracies = []
for ker in ['rbf', 'linear', 'poly']:
  acc = []
  for rdm in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
    clf = SVC(kernel=ker)
    clf.fit(X_train, y_train)
    acc.append(accuracy(clf, X_test, y_test)*100)
  print(f"Kernel: {ker}\t Mean: {np.mean(acc):.2f} \t Std: {np.std(acc):.2f}")

Kernel: rbf	 Mean: 75.33 	 Std: 4.93
Kernel: linear	 Mean: 72.17 	 Std: 4.72
Kernel: poly	 Mean: 72.17 	 Std: 6.01


In [14]:
quantum_accuracies_trained = []
quantum_accuracies_untrained = []
for rdm in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X_minmax,Y, test_size=0.3, stratify=Y, random_state=rdm)
    pred_all_trained = []
    pred_all_untrained = []
    all_params = {}
    init_params = random_params(num_wires=3, num_layers=3)
    init_kernel = lambda x1, x2: kernel(x1, x2, init_params)
    for y1, y2 in combinations(np.unique(Y).numpy(), r=2):
        mask = (y_train==y1) | (y_train==y2)
        y_train_pair = y_train[mask]
        X_train_pair = X_train[mask]
        y_train_pair [y_train_pair == y1] = -1
        y_train_pair [y_train_pair == y2] = 1
        svm_untrained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, init_kernel)).fit(X_train_pair, y_train_pair)
        test_mask = (y_test==y1) | (y_test==y2)
        y_test_pair = y_test[test_mask]
        X_test_pair = X_test[test_mask]
        y_test_pair [y_test_pair == y1] = -1
        y_test_pair [y_test_pair == y2] = 1
        y_pred_pair = svm_untrained.predict(X_test_pair)
        test_accuracy = 1 - np.count_nonzero(y_pred_pair - y_test_pair) / len(y_test_pair)
        print(f"Test Accuracy:{test_accuracy*100:.2f}")
        lr = 2
        new_test_accuracy = test_accuracy
        while (new_test_accuracy != 1) and (new_test_accuracy <= test_accuracy) and (lr >= 0.5):
            if lr >=1 :
                iterations= 500
            else:
                iterations= 1000
            params = all_params.get((y1, y2), init_params)
            params = train(params, X_train_pair, y_train_pair, iterations, 4, lr)
            all_params[(y1, y2)] = params
            trained_kernel = lambda x1, x2: kernel(x1, x2, params)
            svm_trained = SVC(kernel=lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)).fit(X_train_pair, y_train_pair)
            y_pred_pair_new = svm_trained.predict(X_test_pair)
            new_test_accuracy = 1 - np.count_nonzero(y_pred_pair_new - y_test_pair) / len(y_test_pair)
            print(f"Test Accuracy:{new_test_accuracy*100:.2f}")
            lr /= 2
        untrained_predictions = svm_untrained.predict(X_test)
        pred_all_untrained.append((untrained_predictions, (y1,y2)))
        if new_test_accuracy <= test_accuracy:
            pred_all_trained.append((untrained_predictions, (y1,y2)))
        else:
            pred_all_trained.append((svm_trained.predict(X_test), (y1,y2)))
    y_pred_all = []
    for y_pred_pair, labels in pred_all_untrained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_untrained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_untrained.append(final_accuracy_untrained*100)
    print(f"Final Accuracy Untrained {final_accuracy_untrained*100:.2f}")
    y_pred_all = []
    for y_pred_pair, labels in pred_all_trained:
        y_pred_pair[y_pred_pair==-1]=0
        labels = np.array(labels).numpy()
        y_pred_all.append(labels[y_pred_pair])
    y_pred = st.mode(np.vstack(y_pred_all).numpy(),axis=0)[0]
    final_accuracy_trained = 1 - np.count_nonzero(y_pred - y_test) / len(y_test)
    quantum_accuracies_trained.append(final_accuracy_trained*100)
    print(f"Final Accuracy Trained {final_accuracy_trained*100:.2f}")

Test Accuracy:100.00
Test Accuracy:66.67
Step 50 - Alignment = 0.190
Step 100 - Alignment = 0.209
Step 150 - Alignment = 0.214
Step 200 - Alignment = 0.218
Step 250 - Alignment = 0.218
Step 300 - Alignment = 0.225
Step 350 - Alignment = 0.235
Step 400 - Alignment = 0.242
Step 450 - Alignment = 0.239
Step 500 - Alignment = 0.233
Test Accuracy:66.67
Step 50 - Alignment = 0.236
Step 100 - Alignment = 0.238
Step 150 - Alignment = 0.239
Step 200 - Alignment = 0.239
Step 250 - Alignment = 0.236
Step 300 - Alignment = 0.241
Step 350 - Alignment = 0.243
Step 400 - Alignment = 0.237
Step 450 - Alignment = 0.239
Step 500 - Alignment = 0.238
Test Accuracy:75.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.556
Step 100 - Alignment = 0.614
Step 150 - Alignment = 0.604
Step 200 - Alignment = 0.619
Step 250 - Alignment = 0.622
Step 300 - Alignment = 0.623
Step 350 - Alignment =

Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Final Accuracy Untrained 81.67
Final Accuracy Trained 83.33
Test Accuracy:100.00
Test Accuracy:66.67
Step 50 - Alignment = 0.227
Step 100 - Alignment = 0.227
Step 150 - Alignment = 0.231
Step 200 - Alignment = 0.254
Step 250 - Alignment = 0.248
Step 300 - Alignment = 0.268
Step 350 - Alignment = 0.270
Step 400 - Alignment = 0.271
Step 450 - Alignment = 0.271
Step 500 - Alignment = 0.270
Test Accuracy:58.33
Step 50 - Alignment = 0.275
Step 100 - Alignment = 0.277
Step 150 - Alignment = 0.276
Step 200 - Alignment = 0.276
Step 250 - Alignment = 0.273
Step 300 - Alignment = 0.273
Step 350 - Alignment = 0.270
Step 400 - Alignment = 0.265
Step 450 - Alignment = 0.269
Step 500 - Alignment = 0.266
Test Accuracy:58.33
Step 50 - Alignment = 0.263
Step 100 - Alignment = 0.265
Step 150 - Alignment = 0.268
Step 200 - Alignment = 0.268
Step 250 - Alignment = 0.269
Step 300 - Alignment = 0.269
Step 350 - Alignment = 0.271
Step 400 - Align

Step 150 - Alignment = 0.121
Step 200 - Alignment = 0.150
Step 250 - Alignment = 0.150
Step 300 - Alignment = 0.160
Step 350 - Alignment = 0.168
Step 400 - Alignment = 0.176
Step 450 - Alignment = 0.179
Step 500 - Alignment = 0.181
Test Accuracy:83.33
Test Accuracy:100.00
Test Accuracy:83.33
Step 50 - Alignment = 0.235
Step 100 - Alignment = 0.250
Step 150 - Alignment = 0.266
Step 200 - Alignment = 0.273
Step 250 - Alignment = 0.272
Step 300 - Alignment = 0.272
Step 350 - Alignment = 0.270
Step 400 - Alignment = 0.270
Step 450 - Alignment = 0.272
Step 500 - Alignment = 0.274
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.506
Step 100 - Alignment = 0.515
Step 150 - Alignment = 0.511
Step 200 - Alignment = 0.516
Step 250 - Alignment = 0.516
Step 300 - Alignment = 0.518
Step 350 - Alignment = 0.520
Step 400 - Alignment = 0.528
Step 450 - Alignment = 0.532
Step 500 - Alignment = 0.528
Test Accuracy:91.67
Step 50 - Alignment = 0.532
Step 100 - Alignmen

Step 800 - Alignment = 0.684
Step 850 - Alignment = 0.684
Step 900 - Alignment = 0.684
Step 950 - Alignment = 0.684
Step 1000 - Alignment = 0.684
Test Accuracy:91.67
Test Accuracy:91.67
Step 50 - Alignment = 0.575
Step 100 - Alignment = 0.599
Step 150 - Alignment = 0.605
Step 200 - Alignment = 0.610
Step 250 - Alignment = 0.611
Step 300 - Alignment = 0.613
Step 350 - Alignment = 0.613
Step 400 - Alignment = 0.614
Step 450 - Alignment = 0.609
Step 500 - Alignment = 0.613
Test Accuracy:91.67
Step 50 - Alignment = 0.614
Step 100 - Alignment = 0.613
Step 150 - Alignment = 0.615
Step 200 - Alignment = 0.615
Step 250 - Alignment = 0.615
Step 300 - Alignment = 0.615
Step 350 - Alignment = 0.616
Step 400 - Alignment = 0.613
Step 450 - Alignment = 0.614
Step 500 - Alignment = 0.614
Test Accuracy:91.67
Step 50 - Alignment = 0.613
Step 100 - Alignment = 0.616
Step 150 - Alignment = 0.616
Step 200 - Alignment = 0.616
Step 250 - Alignment = 0.616
Step 300 - Alignment = 0.615
Step 350 - Alignment = 

Step 350 - Alignment = 0.347
Step 400 - Alignment = 0.345
Step 450 - Alignment = 0.345
Step 500 - Alignment = 0.345
Step 550 - Alignment = 0.342
Step 600 - Alignment = 0.346
Step 650 - Alignment = 0.345
Step 700 - Alignment = 0.342
Step 750 - Alignment = 0.343
Step 800 - Alignment = 0.344
Step 850 - Alignment = 0.346
Step 900 - Alignment = 0.346
Step 950 - Alignment = 0.341
Step 1000 - Alignment = 0.345
Test Accuracy:75.00
Test Accuracy:75.00
Step 50 - Alignment = 0.185
Step 100 - Alignment = 0.212
Step 150 - Alignment = 0.234
Step 200 - Alignment = 0.231
Step 250 - Alignment = 0.231
Step 300 - Alignment = 0.237
Step 350 - Alignment = 0.219
Step 400 - Alignment = 0.220
Step 450 - Alignment = 0.229
Step 500 - Alignment = 0.221
Test Accuracy:83.33
Test Accuracy:91.67
Step 50 - Alignment = 0.514
Step 100 - Alignment = 0.495
Step 150 - Alignment = 0.510
Step 200 - Alignment = 0.506
Step 250 - Alignment = 0.507
Step 300 - Alignment = 0.514
Step 350 - Alignment = 0.515
Step 400 - Alignment =

...

Step 200 - Alignment = 0.225
Step 250 - Alignment = 0.223
Step 300 - Alignment = 0.227
Step 350 - Alignment = 0.228
Step 400 - Alignment = 0.225
Step 450 - Alignment = 0.227
Step 500 - Alignment = 0.223
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.400
Step 100 - Alignment = 0.409
Step 150 - Alignment = 0.418
Step 200 - Alignment = 0.425
Step 250 - Alignment = 0.422
Step 300 - Alignment = 0.436
Step 350 - Alignment = 0.435
Step 400 - Alignment = 0.444
Step 450 - Alignment = 0.443
Step 500 - Alignment = 0.440
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.493
Step 100 - Alignment = 0.510
Step 150 - Alignment = 0.509
Step 200 - Alignment = 0.507
Step 250 - Alignment = 0.507
Step 300 - Alignment = 0.510
Step 350 - Alignment = 0.515
Step 400 - Alignment = 0.517
Step 450 - Alignment = 0.519
Step 500 - Alignment = 0.517
Test Accuracy:91.67
Step 50 - Alignment = 0.527
Step 100 - Alignment = 0.5

Step 200 - Alignment = 0.647
Step 250 - Alignment = 0.653
Step 300 - Alignment = 0.650
Step 350 - Alignment = 0.651
Step 400 - Alignment = 0.652
Step 450 - Alignment = 0.654
Step 500 - Alignment = 0.660
Test Accuracy:91.67
Step 50 - Alignment = 0.661
Step 100 - Alignment = 0.663
Step 150 - Alignment = 0.664
Step 200 - Alignment = 0.663
Step 250 - Alignment = 0.665
Step 300 - Alignment = 0.665
Step 350 - Alignment = 0.665
Step 400 - Alignment = 0.665
Step 450 - Alignment = 0.666
Step 500 - Alignment = 0.665
Test Accuracy:91.67
Step 50 - Alignment = 0.666
Step 100 - Alignment = 0.665
Step 150 - Alignment = 0.666
Step 200 - Alignment = 0.666
Step 250 - Alignment = 0.665
Step 300 - Alignment = 0.666
Step 350 - Alignment = 0.666
Step 400 - Alignment = 0.665
Step 450 - Alignment = 0.666
Step 500 - Alignment = 0.666
Step 550 - Alignment = 0.663
Step 600 - Alignment = 0.666
Step 650 - Alignment = 0.666
Step 700 - Alignment = 0.666
Step 750 - Alignment = 0.666
Step 800 - Alignment = 0.665
Step 

Step 1000 - Alignment = 0.343
Test Accuracy:83.33
Test Accuracy:100.00
Test Accuracy:100.00
Test Accuracy:83.33
Step 50 - Alignment = 0.454
Step 100 - Alignment = 0.465
Step 150 - Alignment = 0.472
Step 200 - Alignment = 0.477
Step 250 - Alignment = 0.482
Step 300 - Alignment = 0.487
Step 350 - Alignment = 0.488
Step 400 - Alignment = 0.489
Step 450 - Alignment = 0.496
Step 500 - Alignment = 0.500
Test Accuracy:91.67
Test Accuracy:91.67
Step 50 - Alignment = 0.567
Step 100 - Alignment = 0.570
Step 150 - Alignment = 0.569
Step 200 - Alignment = 0.584
Step 250 - Alignment = 0.588
Step 300 - Alignment = 0.590
Step 350 - Alignment = 0.590
Step 400 - Alignment = 0.591
Step 450 - Alignment = 0.591
Step 500 - Alignment = 0.586
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.462
Step 100 - Alignment = 0.466
Step 150 - Alignment = 0.477
Step 200 - Alignment = 0.477
Step 250 - Alignment = 0.476
Step 300 - Alignment = 0.483
Step 350 - Alignment = 0.486
Step 400 - Alignment = 0.48

Step 300 - Alignment = 0.575
Step 350 - Alignment = 0.581
Step 400 - Alignment = 0.582
Step 450 - Alignment = 0.584
Step 500 - Alignment = 0.585
Test Accuracy:83.33
Step 50 - Alignment = 0.586
Step 100 - Alignment = 0.586
Step 150 - Alignment = 0.586
Step 200 - Alignment = 0.585
Step 250 - Alignment = 0.586
Step 300 - Alignment = 0.586
Step 350 - Alignment = 0.586
Step 400 - Alignment = 0.586
Step 450 - Alignment = 0.586
Step 500 - Alignment = 0.586
Step 550 - Alignment = 0.587
Step 600 - Alignment = 0.586
Step 650 - Alignment = 0.586
Step 700 - Alignment = 0.587
Step 750 - Alignment = 0.587
Step 800 - Alignment = 0.588
Step 850 - Alignment = 0.588
Step 900 - Alignment = 0.587
Step 950 - Alignment = 0.589
Step 1000 - Alignment = 0.589
Test Accuracy:83.33
Test Accuracy:100.00
Test Accuracy:91.67
Step 50 - Alignment = 0.605
Step 100 - Alignment = 0.609
Step 150 - Alignment = 0.610
Step 200 - Alignment = 0.611
Step 250 - Alignment = 0.598
Step 300 - Alignment = 0.613
Step 350 - Alignment 

Step 50 - Alignment = 0.654
Step 100 - Alignment = 0.657
Step 150 - Alignment = 0.662
Step 200 - Alignment = 0.659
Step 250 - Alignment = 0.671
Step 300 - Alignment = 0.654
Step 350 - Alignment = 0.673
Step 400 - Alignment = 0.677
Step 450 - Alignment = 0.675
Step 500 - Alignment = 0.679
Test Accuracy:91.67
Step 50 - Alignment = 0.678
Step 100 - Alignment = 0.677
Step 150 - Alignment = 0.679
Step 200 - Alignment = 0.680
Step 250 - Alignment = 0.680
Step 300 - Alignment = 0.680
Step 350 - Alignment = 0.678
Step 400 - Alignment = 0.680
Step 450 - Alignment = 0.680
Step 500 - Alignment = 0.680
Test Accuracy:91.67
Step 50 - Alignment = 0.680
Step 100 - Alignment = 0.679
Step 150 - Alignment = 0.680
Step 200 - Alignment = 0.680
Step 250 - Alignment = 0.680
Step 300 - Alignment = 0.680
Step 350 - Alignment = 0.680
Step 400 - Alignment = 0.680
Step 450 - Alignment = 0.680
Step 500 - Alignment = 0.679
Step 550 - Alignment = 0.679
Step 600 - Alignment = 0.680
Step 650 - Alignment = 0.680
Step 7

In [15]:
np.array(quantum_accuracies_trained) >= np.array(quantum_accuracies_untrained)

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True], requires_grad=True)

In [16]:
np.mean(quantum_accuracies_untrained)

76.0

In [17]:
np.std(quantum_accuracies_untrained)

4.666666666666665

In [18]:
np.mean(quantum_accuracies_trained)

77.66666666666667

In [19]:
np.std(quantum_accuracies_trained)

5.066228051190219